## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/myric/UCF_Projects/Mod_6/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Geraldton,AU,-28.7667,114.6000,63.01,72,40,12.66,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,74.43,75,1,12.12,clear sky
2,2,Kapaa,US,22.0752,-159.3190,82.47,66,62,3.00,broken clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,76.96,83,52,5.01,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,37.47,94,91,3.76,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [4]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Kapaa,US,22.0752,-159.3190,82.47,66,62,3.00,broken clouds
6,6,Jamestown,US,42.0970,-79.2353,81.30,70,76,4.00,broken clouds
7,7,Hilo,US,19.7297,-155.0900,82.26,84,24,3.00,few clouds
22,22,Cabimas,VE,10.3883,-71.4399,87.44,61,77,7.29,broken clouds
24,24,Porbandar,IN,21.6422,69.6093,81.07,76,91,8.55,overcast clouds
25,25,Milledgeville,US,33.0801,-83.2321,88.99,62,97,3.83,overcast clouds
30,30,Albany,US,42.6001,-73.9662,86.67,56,20,6.35,few clouds
32,32,Hithadhoo,MV,-0.6000,73.0833,80.29,76,100,3.40,overcast clouds
45,45,Santa Rosa,PH,14.3122,121.1114,80.58,87,90,3.98,overcast clouds
50,50,Sinnamary,GF,5.3833,-52.9500,80.82,80,90,13.09,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                133
City                   133
Country                133
Lat                    133
Lng                    133
Max Temp               133
Humidity               133
Cloudiness             133
Wind Speed             133
Weather Description    133
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows. 
# No empty rows.

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Kapaa,US,82.47,broken clouds,22.0752,-159.3190,
6,Jamestown,US,81.30,broken clouds,42.0970,-79.2353,
7,Hilo,US,82.26,few clouds,19.7297,-155.0900,
22,Cabimas,VE,87.44,broken clouds,10.3883,-71.4399,
24,Porbandar,IN,81.07,overcast clouds,21.6422,69.6093,
25,Milledgeville,US,88.99,overcast clouds,33.0801,-83.2321,
30,Albany,US,86.67,few clouds,42.6001,-73.9662,
32,Hithadhoo,MV,80.29,overcast clouds,-0.6000,73.0833,
45,Santa Rosa,PH,80.58,overcast clouds,14.3122,121.1114,
50,Sinnamary,GF,80.82,overcast clouds,5.3833,-52.9500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Kapaa,US,82.47,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Jamestown,US,81.30,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Hilo,US,82.26,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
22,Cabimas,VE,87.44,broken clouds,10.3883,-71.4399,Foresgum jejdjdkdkdkdkdkdkdkdkd
24,Porbandar,IN,81.07,overcast clouds,21.6422,69.6093,Lords Inn Porbandar
...,...,...,...,...,...,...,...
674,The Valley,AI,84.06,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
677,Roseburg,US,81.18,clear sky,43.2165,-123.3417,"Holiday Inn Express Roseburg, an IHG Hotel"
683,Aloleng,PH,81.45,overcast clouds,16.1307,119.7824,Steve Walden and Emma Sison-Walden Residence
685,Cabra,ES,80.04,clear sky,37.4725,-4.4421,MS Fuente Las Piedras


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/myric/UCF_Projects/Mod_6/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))